# Markups
Read and Write functions for common markup formats

In [1]:
# | default_exp markup

In [2]:
# | export
# | hide
import json
from json import JSONEncoder
import jsonlines
import yaml

from torch_snippets.loader import BB, L, np
from torch_snippets.paths import *
from torch_snippets.logger import *
import xmltodict
from typing import Union
from torch_snippets.thinc_parser.parser import Config

In [20]:
# | export
# | hide


def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)


_default.default = JSONEncoder().default
JSONEncoder.default = _default


def isnamedtupleinstance(x):
    _type = type(x)
    bases = _type.__bases__
    if len(bases) != 1 or bases[0] != tuple:
        return False
    fields = getattr(_type, "_fields", None)
    if not isinstance(fields, tuple):
        return False
    return all(type(i) == str for i in fields)


def unpack(obj):
    if isinstance(obj, dict):
        return {key: unpack(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [unpack(value) for value in obj]
    elif isnamedtupleinstance(obj):
        return {key: unpack(value) for key, value in obj._asdict().items()}
    elif isinstance(obj, tuple):
        return tuple(unpack(value) for value in obj)
    else:
        return obj


class AttrDict(object):
    """Utility to call a dictionary like an object
    FEATURES:
    0. Get and Set any key (or a nested key) like it's an object, tab-completion supported
    `self.key1.key2[0].key3` ...
    1. Keys and Values are recursively parsed as AttrDicts
    2. Formatting with self.pretty()
    3. Convert everything to dict at any time via `self.to_dict()`
    3. Recursively drop keys using `self.drop(key)` from a json object
    4. Recursively call a function on all values at all levels using `map`

    GOTCHAS:
    1. Due to enforcing a `self.key` format all integer keys are converted to strings implicitly.
    2. You can still call `self[int]` but this internally converts the int to string
    """

    def __init__(self, data):
        for name, value in data.items():
            setattr(self, str(name), self._wrap(value))

    def items(self):
        return self.__dict__.items()

    def keys(self):
        return self.__dict__.keys()

    def values(self):
        return self.__dict__.values()

    def _wrap(self, value):
        if isinstance(value, (L, tuple, list, set, frozenset)):
            value = type(value)([self._wrap(v) for v in value])
            if isinstance(value, (list, L)):
                value = L(value)
            return value
        else:
            return AttrDict(value) if isinstance(value, dict) else value

    __getitem__ = (
        lambda self, x: AttrDict({_x: self[_x] for _x in x})
        if isinstance(x, (list, L))
        else getattr(self, str(x))
    )
    __setitem__ = lambda self, k, v: setattr(self, str(k), self._wrap(v))

    def update(self, dict):
        for k, v in dict.items():
            self[k] = v

    def get(self, key, default=None):
        key = str(key)
        return self[key] if key in self else default

    def __iter__(self):
        return iter(self.keys())

    def __len__(self):
        return len(self.keys())

    def __repr__(self):
        return "{%s}" % str(
            ", ".join("'%s': %s" % (k, repr(v)) for (k, v) in self.__dict__.items())
        )

    def __dir__(self):
        return self.__dict__.keys()

    def __contains__(self, key):
        key = str(key)
        return key in self.__dict__.keys()

    def __delitem__(self, key):
        key = str(key)
        del self.__dict__[key]

    def map(self, func):
        for k in dir(self):
            v = self[k]
            if isinstance(v, AttrDict):
                v.map(func)
            elif isinstance(v, (L, tuple, list, set, frozenset)):
                v = [_v.map(func) if isinstance(_v, AttrDict) else func(_v) for _v in v]
            else:
                v = func(v)
            self[k] = v

    def drop(self, key):
        if key in self:
            del self[key]
        for k in dir(self):
            v = self[k]
            if isinstance(v, AttrDict):
                v.drop(key)
            if isinstance(v, (L, tuple, list, set, frozenset)):
                v = [_v.drop(key) for _v in v if isinstance(_v, AttrDict)]

    def to_dict(self):
        d = {}
        for k in dir(self):
            v = self[k]
            if isinstance(v, AttrDict):
                v = v.to_dict()
            if isinstance(v, (L, tuple, list, set, frozenset)):
                v = [_v.to_dict() if isinstance(_v, AttrDict) else _v for _v in v]
            d[k] = v
        return d

    def pretty(self, print_with_logger=False, *args, **kwargs):
        pretty_json(
            self.to_dict(), print_with_logger=print_with_logger, *args, **kwargs
        )

    def __eq__(self, other):
        return AttrDict(other).to_dict() == self.to_dict()


def pretty_json(
    i, fpath=None, indent=4, print_with_logger=True, return_as_string=False
):
    def set_default(obj):
        if isinstance(obj, (set, BB, L)):
            return list(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, P):
            return str(obj)

    assert isinstance(i, (dict, list))
    i = unpack(i)
    dump = json.dumps(i, indent=indent, default=set_default)
    if fpath:
        makedir(parent(fpath))
        logger.info(f"Dumped a pretty file to {fpath}")
        with open(fpath, "w") as f:
            json.dump(i, f, indent=indent, default=set_default)
            return
    if print_with_logger:
        return logger.opt(depth=1).log("DEBUG", f"\n{dump}")
    else:
        if return_as_string:
            return dump
        print(dump)

In [21]:
from nbdev import show_doc

show_doc(AttrDict)

---

### AttrDict

>      AttrDict (data)

Utility to call a dictionary like an object
FEATURES:
0. Get and Set any key (or a nested key) like it's an object, tab-completion supported
`self.key1.key2[0].key3` ...
1. Keys and Values are recursively parsed as AttrDicts
2. Formatting with self.pretty()
3. Convert everything to dict at any time via `self.to_dict()`
3. Recursively drop keys using `self.drop(key)` from a json object
4. Recursively call a function on all values at all levels using `map`

GOTCHAS:
1. Due to enforcing a `self.key` format all integer keys are converted to strings implicitly.
2. You can still call `self[int]` but this internally converts the int to string

In [22]:
xx = AttrDict({"a": 1, "b": [{"c": 2, "d": 4}, {"e": 3}]})
print(type(xx.b[0]))
print(type(xx.to_dict()["b"][0]))
xx.pretty()

<class '__main__.AttrDict'>

<class 'dict'>

{
    "a": 1,
    "b": [
        {
            "c": 2,
            "d": 4
        },
        {
            "e": 3
        }
    ]
}

In [23]:
x = {"abc": {"b": 10, "c": 11}, "d": {"e": {"f": [2, {"abc": "pqrs"}, 2.234]}}}

y = AttrDict(x)

assert y.abc.b == 10
assert y.d.e.f == [2, {"abc": "pqrs"}, 2.234]

y.d.e.g = 11

# del y.abc.c
# OR
del y["abc"]["c"]

assert y.to_dict() == {
    "abc": {"b": 10},
    "d": {"e": {"f": [2, {"abc": "pqrs"}, 2.234], "g": 11}},
}

y.pretty(indent=2)

assert "abc" in y
assert "def" not in y
print("e" in y.d)

{
  "abc": {
    "b": 10
  },
  "d": {
    "e": {
      "f": [
        2,
        {
          "abc": "pqrs"
        },
        2.234
      ],
      "g": 11
    }
  }
}

True

In [24]:
x = {"abc": {"b": 10, "c": 11}, "d": {"e": {"f": [2, {"abc": "pqrs"}, 2.234]}}}
y = AttrDict(x)
y.abc[["b", "c"]]

{'b': 10, 'c': 11}

In [ ]:
# | export
def read_json(fpath):
    import json

    with open(fpath, "r") as f:
        return json.load(f)


def write_json(obj, fpath, silent=False):
    from datetime import datetime, date

    def set_default(obj):
        if isinstance(obj, set):
            return list(obj)
        if isinstance(obj, (datetime, date)):
            return obj.isoformat()
        raise TypeError

    if not silent:
        logger.opt(depth=1).log("DEBUG", f"Dumping json to {fpath}")
    with open(fpath, "w") as f:
        json.dump(obj, f, indent=4, default=set_default)
    return P(fpath)

In [ ]:
d = [1, {1: 1, 2: 2}, 3]

pretty_json({1: 1, 2: 2})
pretty_json(d)

f = write_json(d, "/tmp/test.json")
print(f)
read_json(f)

[09/21/22 01:15:05] DEBUG                                                                  ]8;id=346895;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_70174/3923293551.py\3923293551.py]8;;\:]8;id=155868;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_70174/3923293551.py#<module>:3\<module>:3]8;;\
                             {                                                                                     
                                 "1": 1,                                                                           
                                 "2": 2                                                                            
                             }                                                                                     

                    DEBUG                                                                  ]8;id=257871;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_70174/3923293551.py\3923293551.py]8;;\:]8;id=817763;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_70174/3923293551.py#<module>:4\<module>:4]8;;\
                             [                                                                                     
                                 1,                                                                                
                                 {                                                                                 
                                     "1": 1,                                                                       
                                     "2": 2                                                                        
                                 },                                                                                
                                 3                                                                                 
                             ]                                                                                     

                    DEBUG    Dumping json to /tmp/test.json                                ]8;id=779794;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_70174/3923293551.py\3923293551.py]8;;\:]8;id=993155;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_70174/3923293551.py#<module>:6\<module>:6]8;;\

/tmp/test.json

[1, {'1': 1, '2': 2}, 3]

In [ ]:
# | export
def write_jsonl(items, dest, mode="w"):
    makedir(parent(dest))
    with jsonlines.open(dest, mode) as writer:
        writer.write_all(items)
        if mode == "a":
            Info(f"Appended {len(items)} items to {dest}")
        elif mode == "w":
            Info(f"Wrote {len(items)} jsons to {dest}")


def read_jsonl(file):
    return [json.loads(line) for line in readlines(file, silent=True)]

In [ ]:
# | export
def read_yaml(file):
    with open(file, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)


def write_yaml(content, fpath):
    with open(fpath, "w") as outfile:
        yaml.dump(content, outfile, default_flow_style=False)

In [ ]:
# | export
def read_xml(file_path: Union[str, P]) -> AttrDict:
    "Read xml data as a dictionary"
    with open(str(file_path)) as xml_file:
        data = AttrDict(xmltodict.parse(xml_file.read()))
    return data


def write_xml(data: Union[AttrDict, dict], file_path: Union[str, P]):
    makedir(parent(file_path))
    "convert a dictionary to xml"
    with open(file_path, "w") as xml_file:
        data = data.to_dict() if isinstance(data, AttrDict) else data
        assert isinstance(data, dict), "Function only supports dicts for now"
        data = xmltodict.unparse(data, pretty=True)
        xml_file.write(data)

In [ ]:
# | export
# | hide
Config = Config